In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('CORP_ITEM_EXP.csv', dtype = {'HSCD':str})

In [3]:
def make_quarer_data(data):
    
    data['EXP_YQ'] = pd.to_datetime(data['EXP_YM'])
    data['YEAR'] = data['EXP_YQ'].dt.year
    data['QUARTER'] = data['EXP_YQ'].dt.quarter
    data['EXP_YQ'] = data['YEAR'].astype(str) + '-' + data['QUARTER'].astype(str) + 'Q' 
    periods = pd.PeriodIndex(year=data['YEAR'], quarter=data['QUARTER'], freq="Q")
    data['QUARTER_INDEX'] = periods 
    
    return data

data = make_quarer_data(data)

In [4]:
data.head()

,BSNO,EXP_YM,CON_EN,CON_KR,HSCD,EXP_AMT,HSCD_LEN,EXP_YQ,YEAR,QUARTER,QUARTER_INDEX
0,5558801211,2021-02-01,United States of America,미국,8481801010,5382.0,10,2021-1Q,2021,1,2021Q1
1,5558801211,2021-02-01,Brazil,브라질,8481801090,77.0,10,2021-1Q,2021,1,2021Q1
2,5558801211,2021-02-01,New Caledonia,뉴칼레도니아,8481801090,643.0,10,2021-1Q,2021,1,2021Q1
3,5558801211,2021-02-01,Netherland,네덜란드,8481801090,9837.0,10,2021-1Q,2021,1,2021Q1
4,5558801211,2021-02-01,New Zealand,뉴질랜드,8481801090,4001.0,10,2021-1Q,2021,1,2021Q1


In [5]:
def make_data_group(data):
    data_group = data.groupby(['QUARTER_INDEX', 'CON_EN', 'HSCD']).agg({'BSNO':'count', 'EXP_AMT':'sum'}).reset_index()
    data_group = data_group[data_group['QUARTER_INDEX'] < '2023Q1']
    return data_group

In [6]:
data_group = make_data_group(data)

In [7]:
def calculate_cqgr(df, country, hscd):
    # filter the DataFrame to only include rows that match the given country and HSCD code
    filtered_df = df[(df['CON_EN'] == country) & (df['HSCD'] == hscd)]

    # group the filtered DataFrame by quarter index and sum the EXP_AMT column
    grouped_df = filtered_df.groupby('QUARTER_INDEX')['EXP_AMT'].sum().reset_index()

    # calculate the quarter-over-quarter growth rate
    grouped_df['QOQ_GROWTH_RATE'] = grouped_df['EXP_AMT'].pct_change()

    # calculate the compound quarter growth rate
    cqgr = (1 + grouped_df['QOQ_GROWTH_RATE']).prod() - 1

    return cqgr

In [8]:
cqgr = calculate_cqgr(data_group, 'Japan', '0101219000')
print(cqgr)

27.285714285714292


In [9]:
def show_data(df, con, hscd):
    data = df[(df['CON_EN'] == con) & (df['HSCD'] == hscd)]
    return data

In [10]:
final_df = data_group.groupby(['CON_EN','HSCD'])[['BSNO', 'EXP_AMT']].mean().reset_index() # 287,247

In [11]:

# define a function to calculate the CQGR for a group
def calculate_cqgr_group(subset):
    # calculate the quarter-over-quarter growth rates
    subset['QOQ_GROWTH_RATE'] = subset['EXP_AMT'].pct_change()

    # calculate the cumulative growth rates for each quarter
    subset['CUMULATIVE_GROWTH_RATE'] = (1 + subset['QOQ_GROWTH_RATE']).cumprod()

    # calculate the final compound quarter growth rate
    cqgr = subset['CUMULATIVE_GROWTH_RATE'].iloc[-1] - 1

    return cqgr

# group the DataFrame by 'CON_EN' and 'HSCD' and apply the function to each group
results_df = data_group.groupby(['CON_EN', 'HSCD']).apply(calculate_cqgr_group).reset_index()

# rename the columns of the results DataFrame
results_df.columns = ['CON_EN', 'HSCD', 'CQGR']


In [12]:
final_df = final_df.merge(results_df, how = "inner")

In [13]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 280394 entries, 0 to 280393
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   CON_EN   280394 non-null  object 
 1   HSCD     280394 non-null  object 
 2   BSNO     280394 non-null  float64
 3   EXP_AMT  280394 non-null  float64
 4   CQGR     195838 non-null  float64
dtypes: float64(3), object(2)
memory usage: 12.8+ MB


In [25]:
final_df[['BSNO', 'EXP_AMT', 'CQGR']] = final_df[['BSNO', 'EXP_AMT', 'CQGR']].round(2) 

In [27]:
final_df.columns = ['CON_EN', 'HSCD', 'AVG_CNT_Q', 'AVG_AMT_Q', 'CQGR']

In [34]:
promision_items_df = final_df[final_df['CQGR'] > 0].reset_index().drop('index', axis =1)

In [115]:
promision_items_df.to_csv('CQGR_PROMISING.csv', index = False)

In [116]:
promision_items_df.head()

,CON_EN,HSCD,AVG_CNT_Q,AVG_AMT_Q,CQGR
0,Afghanistan,1902301010,1.00,29067.50,1.40
1,Afghanistan,2106909099,2.88,239313.75,1.09
2,Afghanistan,3002200000,1.00,2054100.00,39.08
3,Afghanistan,3002410000,1.00,300000.00,4.00
4,Afghanistan,3004399000,1.33,249794.50,2.27


In [117]:
promision_items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99880 entries, 0 to 99879
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   CON_EN     99880 non-null  object 
 1   HSCD       99880 non-null  object 
 2   AVG_CNT_Q  99880 non-null  float64
 3   AVG_AMT_Q  99880 non-null  float64
 4   CQGR       99880 non-null  float64
dtypes: float64(3), object(2)
memory usage: 3.8+ MB


In [ ]:
################# 여기서 부터 재 작성#############

# 1. CQGR_PROMISING.csv 를 불러온다.
# 2. map_df 와 머지한다.
# 3. 국가를 입력하면 유망 품목을 도출하는 함수를 만든다.

In [ ]:
cqgr = pd.read_csv('CQGR_PROMISING.csv', dtype = {"HSCD":str})
cqgr = cqgr.merge(map_df[['HS_CD10', 'HS_NAME_EN', 'MTI_CD6', 'MTI_6NAME', 'MTI_4NAME', 'MTI_3NAME', 'MTI_2NAME', 'MTI_1NAME']], 
                  how = 'left', left_on ='HSCD', right_on = 'HS_CD10').drop('HS_CD10', axis = 1)


In [ ]:
def promising_items_by_country(con):
    result_df = cqgr[cqgr['CON_EN'] == con]
    result_df = result_df[result_df['AVG_CNT_Q'] >= 10]
    # result_df['CQGR'] = result_df['CQGR'].apply(lambda x: '{:.2f}%'.format(x*100))
    
    result_df = result_df.sort_values(by ='CQGR', ascending = False)
    return result_df

In [45]:
map_df = pd.read_csv("HSCD_MTI_MAP_EN.csv", dtype = {"MTI_CD1":str, "MTI_CD2":str, "MTI_CD3":str, "MTI_CD4":str, "MTI_CD6":str, "HS_CD10":str})

In [53]:
map_df.head()

,MTI_CD1,MTI_1NAME,MTI_CD2,MTI_2NAME,MTI_CD3,MTI_3NAME,MTI_CD4,MTI_4NAME,MTI_CD6,MTI_6NAME,HS_CD10,HS_NAME_KR,HS_NAME_EN,HS_CAT
0,0,agricultural & forest & marineproducts,01,agriculturalproducts/farmproduce,011,"cereals,fruits",0111,cereals,011110,rice,1006100000,벼,Rice in the husk (paddy or rough),(쌀)
1,0,agricultural & forest & marineproducts,01,agriculturalproducts/farmproduce,011,"cereals,fruits",0111,cereals,011110,rice,1006201000,메현미,Nonglutinous,(쌀)
2,0,agricultural & forest & marineproducts,01,agriculturalproducts/farmproduce,011,"cereals,fruits",0111,cereals,011110,rice,1006202000,찰현미,Glutinous,(쌀)
3,0,agricultural & forest & marineproducts,01,agriculturalproducts/farmproduce,011,"cereals,fruits",0111,cereals,011110,rice,1006301000,멥쌀,Nonglutinous,(쌀)
4,0,agricultural & forest & marineproducts,01,agriculturalproducts/farmproduce,011,"cereals,fruits",0111,cereals,011110,rice,1006302000,찹쌀,Glutinous,(쌀)


In [85]:
def make_final_result(final_df, map_df):
    promising_df = final_df[final_df['CQGR']  > 0].reset_index().drop('index', axis = 1)
    map_df_en = map_df[['HS_CD10', 'HS_NAME_EN', 'MTI_CD6', 'MTI_6NAME', 'MTI_4NAME', 'MTI_3NAME', 'MTI_2NAME', 'MTI_1NAME']]
    map_df_en.rename(columns = {'HS_CD10':'HSCD'}, inplace = True)
    promising_df = promising_df.merge(map_df_en, how = 'left')
   
    return promising_df
    
    
    

In [108]:

# 국가를 입력받으면 해당 국가의 유망품목을 CQGR 순으로 나오게 하는 함수

def promising_items_by_country(df, con):
    result_df = df[df['CON_EN'] == con]
    result_df = result_df[result_df['AVG_CNT_Q'] >= 10]
    # result_df['CQGR'] = result_df['CQGR'].apply(lambda x: '{:.2f}%'.format(x*100))
    
    result_df = result_df.sort_values(by ='CQGR', ascending = False)
    return result_df
    

In [109]:
result_df = make_final_result(final_df, map_df)

In [110]:
result_df[result_df['CON_EN'] == 'Japan']

,CON_EN,HSCD,AVG_CNT_Q,AVG_AMT_Q,CQGR,HS_NAME_EN,MTI_CD6,MTI_6NAME,MTI_4NAME,MTI_3NAME,MTI_2NAME,MTI_1NAME
39914,Japan,0101219000,2.86,269.71,27.29,NaN,021190,other live stocks,live stock,live animal,live stock rising,agricultural & forest & marineproducts
39915,Japan,0102212000,1.50,32.00,0.11,NaN,021110,bovine,live stock,live animal,live stock rising,agricultural & forest & marineproducts
39916,Japan,0104209000,1.25,78.00,0.75,NaN,021190,other live stocks,live stock,live animal,live stock rising,agricultural & forest & marineproducts
39917,Japan,0106201000,2.25,8517.25,186.50,NaN,021900,other live animal,other live animal,live animal,live stock rising,agricultural & forest & marineproducts
39918,Japan,0106203000,1.00,9779.00,92.03,NaN,021900,other live animal,other live animal,live animal,live stock rising,agricultural & forest & marineproducts
...,...,...,...,...,...,...,...,...,...,...,...,...
42860,Japan,9702903000,1.00,6505.00,1299.00,Lithographs,921000,painting,painting,painting,worksofart,miscellaneousarticles/sundries
42861,Japan,9703001000,3.00,199162.75,1.32,NaN,922000,sculpture,sculpture,sculpture,worksofart,miscellaneousarticles/sundries
42862,Japan,9703901000,3.75,110613.00,5.75,Sculptures,922000,sculpture,sculpture,sculpture,worksofart,miscellaneousarticles/sundries
42863,Japan,9703902000,2.33,34001.33,0.58,Statuary,922000,sculpture,sculpture,sculpture,worksofart,miscellaneousarticles/sundries


In [111]:
jp = promising_items_by_country(result_df, 'Japan')
jp.shape

(676, 12)

In [118]:
promising_items_by_country(result_df, 'Japan').columns

Index(['CON_EN', 'HSCD', 'AVG_CNT_Q', 'AVG_AMT_Q', 'CQGR', 'HS_NAME_EN',
       'MTI_CD6', 'MTI_6NAME', 'MTI_4NAME', 'MTI_3NAME', 'MTI_2NAME',
       'MTI_1NAME'],
      dtype='object')